# Introduction to data analysis in Python

We're going to be working with the Squirrel Census dataset on this course. [The Squirrel Census](https://www.thesquirrelcensus.com/) is a multimedia science, design, and storytelling project focusing on the Eastern gray (Sciurus carolinensis). The dataset contains data for 3,023 squirrel sightings recorded in Central Park, New York in October 2018. It includes information such as location coordinates, age, primary and secondary fur color, elevation, activities, communications, and interactions between squirrels and with humans.

There's a full description of all the columns in this dataset on the [NYC Open Data Portal](https://data.cityofnewyork.us/Environment/2018-Central-Park-Squirrel-Census-Squirrel-Data/vfnx-vebw/about_data) -- have a quick look through the table before continuing and keep it open in a separate tab to refer back to when needed!

## Getting set up

To get started, we need to import some key Python packages:

- The ``pandas`` package is used for working with dataframes (i.e. tabular or structured data)
- The ``numpy`` package is used for working with numerical data (although you may not need it depending on how complex you want to get!)
- The ``matplotlib`` package is used for data visualisation

These packages are all widely used for data analysis. They define new data types that complement the built-in types (like dictionaries and lists), and contain a bunch of useful functions for manipulating and visualising data. The convention is to import each of these packages with a short 'nickname' that makes it quicker and easier to refer to them, as you'll see in the next cell.

In [1]:
# Import the packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

You can get an idea of how much useful stuff is inside these packages by running the following code, which will print out a list of all the different things a named package can do. Many of the entries won't make any sense at this stage -- don't worry! It's just to give you a general idea that there's *a lot* of new data types and functions now at your disposal.

In [2]:
def look_inside(package):
    for option in dir(package):
        print(option)

look_inside(pd) # You could also call this function with np or plt as the "package" argument to see the insides of those packages!

ArrowDtype
BooleanDtype
Categorical
CategoricalDtype
CategoricalIndex
DataFrame
DateOffset
DatetimeIndex
DatetimeTZDtype
ExcelFile
ExcelWriter
Flags
Float32Dtype
Float64Dtype
Grouper
HDFStore
Index
IndexSlice
Int16Dtype
Int32Dtype
Int64Dtype
Int8Dtype
Interval
IntervalDtype
IntervalIndex
MultiIndex
NA
NaT
NamedAgg
Period
PeriodDtype
PeriodIndex
RangeIndex
Series
SparseDtype
StringDtype
Timedelta
TimedeltaIndex
Timestamp
UInt16Dtype
UInt32Dtype
UInt64Dtype
UInt8Dtype
__all__
__builtins__
__cached__
__doc__
__docformat__
__file__
__git_version__
__loader__
__name__
__package__
__path__
__spec__
__version__
_built_with_meson
_config
_is_numpy_dev
_libs
_pandas_datetime_CAPI
_pandas_parser_CAPI
_testing
_typing
_version_meson
annotations
api
array
arrays
bdate_range
compat
concat
core
crosstab
cut
date_range
describe_option
errors
eval
factorize
from_dummies
get_dummies
get_option
infer_freq
interval_range
io
isna
isnull
json_normalize
lreshape
melt
merge
merge_asof
merge_ordered
notna
not

Next we need to import our dataset. If you cloned the git repo for this course and haven't moved anything around, the csv file should just be in the same folder as this notebook. If you get an error in running any of the next few cells, it's probably because that's not the case -- either you just downloaded the notebook and not the dataset, or you've moved things into subfolders.

As you will find with most programming languages, there are a multitude of ways in which you can load in data. Loading data efficiently is a critical step in any data analysis process, and while Python's built-in functions can handle basic data loading tasks, the pandas library offers a more powerful and flexible approach. Run the next cell to load the squirrels dataset using pandas.

In [3]:
data = pd.read_csv("squirrels.csv")

Note that pandas also has functions to import different types of datasets, like Excel files (which end with ``.xlsx``), JSON files, and SQL databases.

There's also many other options within the ``read_csv`` function that we're not using here. For example:

- If you have data with no header row (i.e. no column names, the data itself starts on the first row), you can specify ``header=None`` and then give a list of column names like this: ``names=["col1", "col2", "col3"]`` (you can also specify ``header=0`` and then give a list of names to overwrite the existing column names).
- If missing values in your dataset are represented in some way other than "NA", "NaN" or just an empty cell, you can use the ``na_values`` parameter to tell Python what other values to treat as NA.
- If you're working with a particularly messy dataset, there are other options you can use to deal with errors in loading data, like ``on_bad_lines="skip"``, or specifying an ``encoding`` to help Python read unusual characters.

We won't go through any of these advanced options on this course, but you can always have a look at the [documentation](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html) if there's something specific you want to achieve.

## Getting started with pandas dataframes

Now that we've read our dataset in, we need to understand how it's actually stored in Python. When we use pandas to load data, that data gets stored as a pandas DataFrame, which is a new data type. DataFrames are two-dimensional tabular data structures i.e. they have rows and columns. First, let's create a small DataFrame from scratch to see what they look like.

In [4]:
cdcs = pd.DataFrame({
    "name": ["Lucia", "Aislinn", "Sarah"],
    "role": ["Training Manager", "Training Fellow", "Training Fellow"],
    "PhD": ["Archaeology", "Linguistics", "Data visualisation"],
    "year_submitted": [2018, 2025, 2024],
})

cdcs

,name,role,PhD,year_submitted
0,Lucia,Training Manager,Archaeology,2018
1,Aislinn,Training Fellow,Linguistics,2025
2,Sarah,Training Fellow,Data visualisation,2024


You may notice that what we pass to the ``pd.DataFrame`` function looks just like a Python dictionary, with strings as keys and lists of items as values. And what it spits out is a nice tidy table with named columns and rows. The column names are the dictionary keys, and the cells are filled in with the dictionary values in the order we specified them.

To create a DataFrame like this, all the lists of items to go in the different columns must be the same length. Let's see what happens when we don't do that.

In [ ]:
# 3 items each for the name, role and PhD columns, but only 2 values for the year_submitted column
pd.DataFrame({
    "name": ["Lucia", "Aislinn", "Sarah"],
    "role": ["Training Manager", "Training Fellow", "Training Fellow"],
    "PhD": ["Archaeology", "Linguistics", "Data visualisation"],
    "year_submitted": [2018, 2025],
})

Ok, that's a big error message, but the important bit is right at the bottom!

In practice, it's fairly unusual to create DataFrames like this -- usually we'll be reading in an existing dataset like the squirrels one we're using on this course. So let's move on to that now.

We can see what our real dataset looks like by running the following cell, which will show the first few rows (by default, 5 rows, but you can put a number in the brackets if you want to see more/fewer).

In [5]:
# Show the first few rows of the dataframe
data.head()

,X,Y,Unique Squirrel ID,Hectare,Shift,Date,Hectare Squirrel Number,Age,Primary Fur Color,Highlight Fur Color,...,Kuks,Quaas,Moans,Tail flags,Tail twitches,Approaches,Indifferent,Runs from,Other Interactions,Lat/Long
0,-73.956134,40.794082,37F-PM-1014-03,37F,PM,10142018,3,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,NaN,POINT (-73.9561344937861 40.7940823884086)
1,-73.968857,40.783783,21B-AM-1019-04,21B,AM,10192018,4,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,NaN,POINT (-73.9688574691102 40.7837825208444)
2,-73.974281,40.775534,11B-PM-1014-08,11B,PM,10142018,8,NaN,Gray,NaN,...,False,False,False,False,False,False,False,False,NaN,POINT (-73.97428114848522 40.775533619083)
3,-73.959641,40.790313,32E-PM-1017-14,32E,PM,10172018,14,Adult,Gray,NaN,...,False,False,False,False,False,False,False,True,NaN,POINT (-73.9596413903948 40.7903128889029)
4,-73.970268,40.776213,13E-AM-1017-05,13E,AM,10172018,5,Adult,Gray,Cinnamon,...,False,False,False,False,False,False,False,False,NaN,POINT (-73.9702676472613 40.7762126854894)


In [6]:
# Or the last few rows!
data.tail()

,X,Y,Unique Squirrel ID,Hectare,Shift,Date,Hectare Squirrel Number,Age,Primary Fur Color,Highlight Fur Color,...,Kuks,Quaas,Moans,Tail flags,Tail twitches,Approaches,Indifferent,Runs from,Other Interactions,Lat/Long
3018,-73.963943,40.790868,30B-AM-1007-04,30B,AM,10072018,4,Adult,Gray,NaN,...,False,False,False,False,False,False,False,True,NaN,POINT (-73.9639431360458 40.7908677445466)
3019,-73.970402,40.782560,19A-PM-1013-05,19A,PM,10132018,5,Adult,Gray,White,...,False,False,False,False,False,False,True,False,NaN,POINT (-73.9704015859639 40.7825600069973)
3020,-73.966587,40.783678,22D-PM-1012-07,22D,PM,10122018,7,Adult,Gray,"Black, Cinnamon, White",...,False,False,False,False,False,False,True,False,NaN,POINT (-73.9665871993517 40.7836775064883)
3021,-73.963994,40.789915,29B-PM-1010-02,29B,PM,10102018,2,NaN,Gray,"Cinnamon, White",...,False,False,False,False,False,False,True,False,NaN,POINT (-73.9639941227864 40.7899152327912)
3022,-73.975479,40.769640,5E-PM-1012-01,05E,PM,10122018,1,Adult,Cinnamon,"Gray, White",...,False,False,False,False,False,True,False,False,NaN,POINT (-73.9754794191553 40.7696404489025)


We can also find out the total size of our dataset using ``size`` and ``shape``. See if you can figure out the difference between these by running the next two cells!

In [7]:
data.size

93713

In [8]:
data.shape

(3023, 31)

There a variety of other methods we can use to explore the structure of a DataFrame. For example, ``data.columns`` will show us the column names, ``data.index`` will show us the row labels, and ``data.dtypes`` will show us what data type is stored in each column. 

For a nice overview, we can use the ``info`` method. This provides a concise summary of the DataFrame, including the number of non-null entries and data types of each column. The ``object`` dtype refers both to strings and to columns with mixed data types.

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3023 entries, 0 to 3022
Data columns (total 31 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   X                                           3023 non-null   float64
 1   Y                                           3023 non-null   float64
 2   Unique Squirrel ID                          3023 non-null   object 
 3   Hectare                                     3023 non-null   object 
 4   Shift                                       3023 non-null   object 
 5   Date                                        3023 non-null   int64  
 6   Hectare Squirrel Number                     3023 non-null   int64  
 7   Age                                         2902 non-null   object 
 8   Primary Fur Color                           2968 non-null   object 
 9   Highlight Fur Color                         1937 non-null   object 
 10  Combination 

If you want to access a specific column, you can index the DataFrame using the column name, as follows:

In [10]:
data["Primary Fur Color"]

0            NaN
1            NaN
2           Gray
3           Gray
4           Gray
          ...   
3018        Gray
3019        Gray
3020        Gray
3021        Gray
3022    Cinnamon
Name: Primary Fur Color, Length: 3023, dtype: object

This returns a Series, which is another data type that's similar to a list but with "axis labels" (i.e. row numbers). You can also get all the values from one column as a normal list by wrapping the code above in ``list()``.

In [11]:
list(data["Primary Fur Color"])

[nan,
 nan,
 'Gray',
 'Gray',
 'Gray',
 'Cinnamon',
 'Gray',
 'Gray',
 'Gray',
 'Gray',
 'Gray',
 'Gray',
 'Gray',
 'Cinnamon',
 'Gray',
 'Gray',
 'Gray',
 'Gray',
 'Gray',
 'Gray',
 'Cinnamon',
 'Cinnamon',
 'Gray',
 'Gray',
 'Gray',
 'Gray',
 'Gray',
 'Gray',
 'Gray',
 'Gray',
 'Cinnamon',
 'Gray',
 'Gray',
 'Gray',
 'Gray',
 'Gray',
 'Gray',
 'Black',
 'Gray',
 'Gray',
 'Gray',
 'Gray',
 'Gray',
 'Gray',
 'Gray',
 'Black',
 'Gray',
 'Gray',
 'Gray',
 'Gray',
 'Gray',
 'Gray',
 'Gray',
 'Gray',
 'Gray',
 'Gray',
 'Gray',
 'Gray',
 'Cinnamon',
 'Gray',
 'Gray',
 'Gray',
 'Cinnamon',
 'Cinnamon',
 'Gray',
 'Gray',
 'Gray',
 'Gray',
 'Gray',
 'Gray',
 'Gray',
 'Gray',
 'Gray',
 'Gray',
 'Cinnamon',
 'Gray',
 'Gray',
 'Gray',
 'Cinnamon',
 'Gray',
 'Gray',
 'Gray',
 'Cinnamon',
 'Cinnamon',
 'Gray',
 'Gray',
 'Gray',
 nan,
 'Gray',
 'Gray',
 'Gray',
 'Gray',
 'Gray',
 'Gray',
 'Black',
 'Gray',
 'Gray',
 'Gray',
 'Gray',
 'Gray',
 'Gray',
 'Gray',
 'Gray',
 'Gray',
 'Gray',
 'Gray',
 'Gr

To select multiple columns, you can pass a list of column names. This will return a DataFrame with the selected columns. Note that the columns don't have to be in the same order as they were in the original DataFrame!

In [12]:
data[["Unique Squirrel ID", "Primary Fur Color", "Date"]]

,Unique Squirrel ID,Primary Fur Color,Date
0,37F-PM-1014-03,NaN,10142018
1,21B-AM-1019-04,NaN,10192018
2,11B-PM-1014-08,Gray,10142018
3,32E-PM-1017-14,Gray,10172018
4,13E-AM-1017-05,Gray,10172018
...,...,...,...
3018,30B-AM-1007-04,Gray,10072018
3019,19A-PM-1013-05,Gray,10132018
3020,22D-PM-1012-07,Gray,10122018
3021,29B-PM-1010-02,Gray,10102018


To select a particular row by row number, you can use ``iloc``. If you have a dataset with named rows instead of numbered rows, you can use ``loc`` to select rows by name.

In [13]:
data.iloc[[10]]

,X,Y,Unique Squirrel ID,Hectare,Shift,Date,Hectare Squirrel Number,Age,Primary Fur Color,Highlight Fur Color,...,Kuks,Quaas,Moans,Tail flags,Tail twitches,Approaches,Indifferent,Runs from,Other Interactions,Lat/Long
10,-73.969506,40.782351,20B-PM-1013-05,20B,PM,10132018,5,Adult,Gray,White,...,False,False,False,False,False,False,True,False,NaN,POINT (-73.9695063535333 40.7823507678183)


To select a continguous range of rows, you can use list slicing syntax.

In [14]:
data.iloc[10:20]

,X,Y,Unique Squirrel ID,Hectare,Shift,Date,Hectare Squirrel Number,Age,Primary Fur Color,Highlight Fur Color,...,Kuks,Quaas,Moans,Tail flags,Tail twitches,Approaches,Indifferent,Runs from,Other Interactions,Lat/Long
10,-73.969506,40.782351,20B-PM-1013-05,20B,PM,10132018,5,Adult,Gray,White,...,False,False,False,False,False,False,True,False,NaN,POINT (-73.9695063535333 40.7823507678183)
11,-73.964003,40.782031,22F-PM-1014-06,22F,PM,10142018,6,Adult,Gray,NaN,...,False,False,False,False,True,False,True,False,NaN,POINT (-73.9640032826529 40.7820309825448)
12,-73.953217,40.791967,36I-PM-1007-01,36I,PM,10072018,1,Adult,Gray,Cinnamon,...,False,False,False,False,False,False,True,False,NaN,POINT (-73.9532170504865 40.7919669739962)
13,-73.976860,40.770280,5C-PM-1010-09,05C,PM,10102018,9,Adult,Cinnamon,Gray,...,False,False,False,False,False,False,False,False,NaN,POINT (-73.9768603630674 40.7702795904962)
14,-73.970611,40.769812,7H-AM-1006-05,07H,AM,10062018,5,Adult,Gray,Cinnamon,...,False,False,False,False,False,False,True,False,NaN,POINT (-73.9706105896967 40.7698124821507)
15,-73.970378,40.778753,16C-PM-1018-03,16C,PM,10182018,3,Adult,Gray,"Cinnamon, White",...,False,False,False,False,False,False,True,False,NaN,POINT (-73.9703781726172 40.7787526130321)
16,-73.970393,40.776503,14E-AM-1008-23,14E,AM,10082018,23,Adult,Gray,NaN,...,False,False,False,False,False,False,True,False,NaN,POINT (-73.9703925210471 40.7765032004992)
17,-73.963818,40.792417,32A-PM-1013-03,32A,PM,10132018,3,Juvenile,Gray,Cinnamon,...,False,False,False,False,False,False,True,False,NaN,POINT (-73.9638179439747 40.7924173263904)
18,-73.958407,40.791381,33F-AM-1008-01,33F,AM,10082018,1,Adult,Gray,NaN,...,False,False,False,False,False,False,True,False,NaN,POINT (-73.9584070974734 40.7913812490557)
19,-73.967113,40.778486,17F-AM-1007-07,17F,AM,10072018,7,Adult,Gray,White,...,False,False,False,True,False,False,True,False,NaN,POINT (-73.9671130680114 40.7784859700171)


Probably more helpfully than selecting rows by index, you can also filter based on certain criteria. For example, if you just want to see the squirrels that were recorded as being adults, you can do:

In [15]:
# Filter the dataframe
adult_squirrels = data[data["Age"] == "Adult"]

# See how many rows that leaves us with
adult_squirrels.shape

(2568, 31)

You can also use more complex criteria, like:

In [16]:
morning_adult_squirrels = data[(data["Age"] == "Adult") & (data["Shift"] == "AM")]
morning_adult_squirrels.shape

(1152, 31)

Have a play with these functions -- try creating some new DataFrames that contain a particular subset of the squirrels data, or creating a new DataFrame from scratch and then seeing what properties it has based on the type of data you've entered in each column. N.B. pandas DataFrames can contain all kinds of data -- in the toy example earlier we just saw strings and integers, but trying adding some floats, booleans (True or False) etc.

## Data wrangling

When we start working with a dataset like our squirrels one, we often have to do a lot of work to get it from its raw format into a format that's more useable for the analysis we want to do: this process is known as "data wrangling" (also sometimes "data munging", but I'll leave you to decide which term sounds nicer!). 

The exact steps you'll need to go through will depend on the properties of your dataset, but here's some common ones:

- Dealing with missing data and NA values (more on this later!).
- Cleaning text columns by removing leading/trailing whitespace, converting all characters to lowercase, removing punctuation etc.
- Removing duplicate rows/columns.
- Concatenating values from different columns to make a composite measure (the "Combination of Primary and Highlight Color" column in the squirrels dataset is an example of this).
- Splitting a column out into its component parts if it's already composite.
- Discretising continuous values into categories. For example, if we had a dataset of humans with a column for Age (in years), we might want to make a new column that labelled each person as "child" if they were under 18 or "adult" if they were over 18.
- Moving columns around (never strictly necessary for analysis purposes, but can help make things more human readable to have related columns next to each other).

The squirrels dataset is already quite well cleaned, so some of these operations won't really do anything, but we can still see how they work so you can apply them to other datasets you're working with in the future!

### Text cleaning

Let's have a look at text cleaning first. Whitespace at the beginning or end of strings can cause issues in data analysis (because Python will treat "string", " string" and "string " as three different things) -- to remove it, we can use the ``str.strip()`` method. 

In [17]:
# Remove whitespace from the Primary Fur Color column
data["Primary Fur Color"] = data["Primary Fur Color"].str.strip()

# It's standard practice to always look at the first few rows of a dataframe after applying any transformation to make sure it's worked!
data.head() 

,X,Y,Unique Squirrel ID,Hectare,Shift,Date,Hectare Squirrel Number,Age,Primary Fur Color,Highlight Fur Color,...,Kuks,Quaas,Moans,Tail flags,Tail twitches,Approaches,Indifferent,Runs from,Other Interactions,Lat/Long
0,-73.956134,40.794082,37F-PM-1014-03,37F,PM,10142018,3,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,NaN,POINT (-73.9561344937861 40.7940823884086)
1,-73.968857,40.783783,21B-AM-1019-04,21B,AM,10192018,4,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,NaN,POINT (-73.9688574691102 40.7837825208444)
2,-73.974281,40.775534,11B-PM-1014-08,11B,PM,10142018,8,NaN,Gray,NaN,...,False,False,False,False,False,False,False,False,NaN,POINT (-73.97428114848522 40.775533619083)
3,-73.959641,40.790313,32E-PM-1017-14,32E,PM,10172018,14,Adult,Gray,NaN,...,False,False,False,False,False,False,False,True,NaN,POINT (-73.9596413903948 40.7903128889029)
4,-73.970268,40.776213,13E-AM-1017-05,13E,AM,10172018,5,Adult,Gray,Cinnamon,...,False,False,False,False,False,False,False,False,NaN,POINT (-73.9702676472613 40.7762126854894)


Now this DataFrame has 31 columns, which is quite a few, but many datasets you'll be working with might have lots more than that! It's going to get pretty inefficient to do this on a column-by-column basis. There is a way to apply the same transformation to all columns containing text data:

In [18]:
data[data.select_dtypes(include=["object"]).columns] = data.select_dtypes(include=["object"]).apply(lambda x: x.str.strip())
data.head() 

,X,Y,Unique Squirrel ID,Hectare,Shift,Date,Hectare Squirrel Number,Age,Primary Fur Color,Highlight Fur Color,...,Kuks,Quaas,Moans,Tail flags,Tail twitches,Approaches,Indifferent,Runs from,Other Interactions,Lat/Long
0,-73.956134,40.794082,37F-PM-1014-03,37F,PM,10142018,3,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,NaN,POINT (-73.9561344937861 40.7940823884086)
1,-73.968857,40.783783,21B-AM-1019-04,21B,AM,10192018,4,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,NaN,POINT (-73.9688574691102 40.7837825208444)
2,-73.974281,40.775534,11B-PM-1014-08,11B,PM,10142018,8,NaN,Gray,NaN,...,False,False,False,False,False,False,False,False,NaN,POINT (-73.97428114848522 40.775533619083)
3,-73.959641,40.790313,32E-PM-1017-14,32E,PM,10172018,14,Adult,Gray,NaN,...,False,False,False,False,False,False,False,True,NaN,POINT (-73.9596413903948 40.7903128889029)
4,-73.970268,40.776213,13E-AM-1017-05,13E,AM,10172018,5,Adult,Gray,Cinnamon,...,False,False,False,False,False,False,False,False,NaN,POINT (-73.9702676472613 40.7762126854894)


This code looks pretty horrible! But let's step through it, starting on the right of the equals sign:

- ``data.select_dtypes(include=['object'])`` selects all text columns in the DataFrame (columns with type object).
- ``.apply(lambda x: x.str.strip())`` applies ``strip()`` to each of those columns. ``lambda`` is what's called an "anonymous function" -- don't worry too much about this, but do ask if you want to know more!
- The left hand side of the equals sign tells Python to store the result of that operation in each of the columns that are of type object (if you didn't include this, Python would carry out the operation and show you the result, but it wouldn't actually be saved in the DataFrame).

It's also common to convert all text to lowercase (because, again, "String" and "string" are treated differently, so we want to make sure things are standardised before we start our analysis). To do this, we can use the ``str.lower()`` function in the same way as ``str.strip()``. Try doing this with just one named column or applying it to all the text columns at once. N.B. You can also chain operations (i.e. ``str.strip().str.lower()``)!

### Dealing with duplicate data

Duplicate data can cause inaccuracies in analysis and results. It is important to identify and handle duplicates to ensure data integrity. Duplicates can occur in both rows and columns, and handling them involves either removing or aggregating the duplicate entries.

Identifying duplicate rows is the first step in handling duplicate data. We can use the ``duplicated()`` method to find duplicate rows in the dataset. The `duplicated()` method returns a boolean Series indicating whether each row is a duplicate. We sum this Series to get the total number of duplicate rows and display the actual duplicate rows for inspection.

In [19]:
# Check for duplicate rows
duplicate_rows = data.duplicated()

# Display the count of duplicate rows
print(f'Total duplicate rows: {duplicate_rows.sum()}')

# Show the duplicate rows
print(data[duplicate_rows])

Total duplicate rows: 0
Empty DataFrame
Columns: [X, Y, Unique Squirrel ID, Hectare, Shift, Date, Hectare Squirrel Number, Age, Primary Fur Color, Highlight Fur Color, Combination of Primary and Highlight Color, Color notes, Location, Above Ground Sighter Measurement, Specific Location, Running, Chasing, Climbing, Eating, Foraging, Other Activities, Kuks, Quaas, Moans, Tail flags, Tail twitches, Approaches, Indifferent, Runs from, Other Interactions, Lat/Long]
Index: []

[0 rows x 31 columns]


Ok, like I said, it's a pretty clean dataset, so we don't have any duplicates! But running the following code won't do any harm, and let's you see how you'd use it in a case where you did have duplicates.

Once duplicates are identified, we can remove them using the `drop_duplicates()` method. This method removes duplicate rows, keeping the first occurrence by default.

In [20]:
data = data.drop_duplicates()

Duplicate columns can also occur, especially when merging datasets. We can identify duplicate columns by comparing the values across columns.

In [21]:
# Check for duplicate columns by transposing the dataframe and using duplicated
duplicate_columns = data.T.duplicated()

# Display the duplicate columns
print(f'Duplicate columns: {duplicate_columns.sum()}')
print(data.T[duplicate_columns].T.head())

Duplicate columns: 0
Empty DataFrame
Columns: []
Index: [0, 1, 2, 3, 4]


We transpose the DataFrame and use the ``duplicated()`` method to identify duplicate columns. Transposing the DataFrame switches rows and columns, allowing us to use the same method for detecting duplicates.

After identifying duplicate columns, we can remove them by selecting only the unique columns.

In [22]:
data = data.loc[:, ~data.T.duplicated()]

We remove duplicate columns by transposing the DataFrame, removing the duplicate columns, and transposing it back. This ensures that we retain only unique columns in the dataset.

Sometimes, rows may not be exact duplicates but may still need to be handled due to partial duplication. We can use the `drop_duplicates()` method with specific columns to handle this, by passing a list of columns to the `subset` parameter to specify which columns to consider when identifying duplicates. This allows us to handle partial duplicates based on specific columns.

In [23]:
unique_data = data.drop_duplicates(subset=["Unique Squirrel ID"]) # Maybe we only want to keep one row per squirrel
unique_data.shape

(3018, 31)

### Transforming data into different types

Another thing I noticed about the squirrels dataset is that the Date column is just being treated as a number, not a real date -- if you look at the [NYC Open Data Portal](https://data.cityofnewyork.us/Environment/2018-Central-Park-Squirrel-Census-Squirrel-Data/vfnx-vebw/about_data) you'll see that this column is a "concatenation of the sighting session day and month". Python has a package, ``datetime``, which is specifically designed to work with dates, so we might want to transform this column into that data type. Let's give it a go!

In [24]:
# First we need to import the datetime package
import datetime as dt

# Then we can add a new column where we convert the date column into a datetime object
# The format argument tells Python how the elements are ordered in the original column (this is an American dataset so it's MM/DD/YYYY!)
data["parsed_date"] = pd.to_datetime(data["Date"], format="%m%d%Y")

# And then if we have a look at info() again we'll see the new column at the bottom with the datetime dytpe
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3023 entries, 0 to 3022
Data columns (total 32 columns):
 #   Column                                      Non-Null Count  Dtype         
---  ------                                      --------------  -----         
 0   X                                           3023 non-null   float64       
 1   Y                                           3023 non-null   float64       
 2   Unique Squirrel ID                          3023 non-null   object        
 3   Hectare                                     3023 non-null   object        
 4   Shift                                       3023 non-null   object        
 5   Date                                        3023 non-null   int64         
 6   Hectare Squirrel Number                     3023 non-null   int64         
 7   Age                                         2902 non-null   object        
 8   Primary Fur Color                           2968 non-null   object        
 9   Highligh

### Other tasks you can think of!

Have a think about some other changes or additions you'd like to make to this dataset (or any others you've brought to work on). Discuss with your partner/others at your table how you think you might go about implementing these. If you feel confident navigating the pandas documentation yourself, have a go at running your own code in the next few cells!

## Dealing with missing data

You may have noticed when we ran ``data.info()`` that there's quite a lot of missing data in some columns; for example, that output told us that there were only 182 entries in the "color notes" column out of a total of 3023 rows.

Another way to look at this is to count how many missing values are in each column by chaining two different operations. ``isnull()`` returns a DataFrame of the same shape as the original, but with boolean values indicating the presence of missing data (try removing ``.sum()`` to see what this looks like!). Chaining the ``sum()`` method allows us to count the number of missing values in each column, which gives a clear overview of where data is missing. 

In [25]:
data.isnull().sum()

X                                                0
Y                                                0
Unique Squirrel ID                               0
Hectare                                          0
Shift                                            0
Date                                             0
Hectare Squirrel Number                          0
Age                                            121
Primary Fur Color                               55
Highlight Fur Color                           1086
Combination of Primary and Highlight Color       0
Color notes                                   2841
Location                                        64
Above Ground Sighter Measurement               114
Specific Location                             2547
Running                                          0
Chasing                                          0
Climbing                                         0
Eating                                           0
Foraging                       

There's no one way to deal with missing data. Sometimes, the best thing to do will be to just remove certain rows or columns, but other times, you'll want to try and fill in missing values. Each approach has trade-offs, and the best method depends on the specific dataset and analysis requirements.

We can use the ``dropna()`` function to remove all rows or columns with missing values at once, but based on the summary above, this might not leave us with much data!

In [26]:
# Remove all rows with any missing values
data_dropped_rows = data.dropna()

# Remove all columns with any missing values
data_dropped_columns = data.dropna(axis=1) # axis=1 tells dropna() to go by columns instead of rows (its default behaviour)

# Check how many rows this leaves us with
print("Original dataset shape:", data.shape)
print("After dropping rows:", data_dropped_rows.shape)
print("After dropping columns:", data_dropped_columns.shape)

Original dataset shape: (3023, 32)
After dropping rows: (1, 32)
After dropping columns: (3023, 23)


Just dropping rows/columns where there's **any** missing data seems perhaps a bit extreme in this case. Instead, I think I'll just remove a few of the columns that have a lot of missing data using ``drop()``, and fill in some others with a default value using ``fillna()``.

In [27]:
# Drop columns with a lot of missing data
# N.B. inplace=True means the result of this operation will be saved in the dataframe without us explicitly assigning it using data = data.drop()
data.drop(columns=["Color notes", "Specific Location", "Other Activities", "Other Interactions"], inplace=True)

# Check how many columns we've got now (should be less than before!)
data.shape

(3023, 28)

In [28]:
# Fill missing cells in text columns with "unknown"
data[data.select_dtypes(include=["object"]).columns] = data.select_dtypes(include=["object"]).fillna("unknown")

# Visually inspect the data (we should see "unknown" in some cells now since we had NA values on the first few rows before)
data.head()

,X,Y,Unique Squirrel ID,Hectare,Shift,Date,Hectare Squirrel Number,Age,Primary Fur Color,Highlight Fur Color,...,Kuks,Quaas,Moans,Tail flags,Tail twitches,Approaches,Indifferent,Runs from,Lat/Long,parsed_date
0,-73.956134,40.794082,37F-PM-1014-03,37F,PM,10142018,3,unknown,unknown,unknown,...,False,False,False,False,False,False,False,False,POINT (-73.9561344937861 40.7940823884086),2018-10-14
1,-73.968857,40.783783,21B-AM-1019-04,21B,AM,10192018,4,unknown,unknown,unknown,...,False,False,False,False,False,False,False,False,POINT (-73.9688574691102 40.7837825208444),2018-10-19
2,-73.974281,40.775534,11B-PM-1014-08,11B,PM,10142018,8,unknown,Gray,unknown,...,False,False,False,False,False,False,False,False,POINT (-73.97428114848522 40.775533619083),2018-10-14
3,-73.959641,40.790313,32E-PM-1017-14,32E,PM,10172018,14,Adult,Gray,unknown,...,False,False,False,False,False,False,False,True,POINT (-73.9596413903948 40.7903128889029),2018-10-17
4,-73.970268,40.776213,13E-AM-1017-05,13E,AM,10172018,5,Adult,Gray,Cinnamon,...,False,False,False,False,False,False,False,False,POINT (-73.9702676472613 40.7762126854894),2018-10-17


In [29]:
# And check whether we still have any null values
data.isnull().sum()

X                                             0
Y                                             0
Unique Squirrel ID                            0
Hectare                                       0
Shift                                         0
Date                                          0
Hectare Squirrel Number                       0
Age                                           0
Primary Fur Color                             0
Highlight Fur Color                           0
Combination of Primary and Highlight Color    0
Location                                      0
Above Ground Sighter Measurement              0
Running                                       0
Chasing                                       0
Climbing                                      0
Eating                                        0
Foraging                                      0
Kuks                                          0
Quaas                                         0
Moans                                   

We'll see another possible method of dealing with missing data next week when we get on to summary statistics: filling empty cells with a computed statistic (e.g. the mean of the column).